# 예측 결과 출력
* predictionResult 변수에 Str 타입으로 예측결과를 입력해주세요.
* 그 이후 순서대로 실행하시면 됩니다.

순서
1. 이미지 로드(샘플)
2. 예측 결과에서 이미지 로드

In [1]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from operator import itemgetter

# 한글 출력준비

* 네이버 나눔고딕 폰트 등을 설치 또는 사용
* 폰트를 새로 설치 했다면 아래 실행
    ```
    import matplotlib as mpl
    mpl.font_manager._rebuild()
    ```

* 한글 폰트 사용하려면 관련 폰트를 등록하면 된다. (나의 경우는 나눔코딩 폰트)
    ```
    path = 'c:\\windows\\fonts\\nanumgothiccoding.ttf'
    font_name = fm.FontProperties(fname=path, size=50).get_name()
    plt.rc('font', family=font_name)
    ```


In [2]:
path = 'c:\\windows\\fonts\\nanumgothiccoding.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
plt.rc('font', family=font_name)

# 예측 결과 로딩

In [3]:
def load_pred_result(file_path):
    pred_dic = {}
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)        
        for row in reader:
            history = row[1].split('-')
            preds = list(map(lambda x: x.split('/'), row[2].split(':')))
            pred_dic[row[0]] = (history, preds)            
    return pred_dic

In [4]:
def load_item_b(file_path):
    item_b= []
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)        
        for row in reader:
            item_b.append(row)
    return item_b

# Elastic Search 를 이용

In [5]:
es = elasticsearch.Elasticsearch('twiceSpark1:9200')

def es_search(es, deal_ids):
    res = es.search(index='ojm5', body={'query': {'ids':{'values':deal_ids}}}, size=100)
    n = res['hits']['total']
    dic = {}
    for hit in res['hits']['hits']:
        dic[hit['_id']] =  hit['_source']
    return dic
    

In [6]:
with open(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model\deal_to_tn.pkl', 'rb') as base_file:
    deal_to_tn1 = pickle.load(base_file)

In [7]:
def show_preds(pred):
    img_base_url=r'http://img.wemep.co.kr/'
    history = es_search(es, pred[0])
        
    fig, axes = plt.subplots(ncols=len(pred[0]), figsize=(16,16))
    
    for i, h in enumerate(pred[0]):
        title = history[h]['_2']
        img = history[h]['_3']
        f = urllib.request.urlopen(img_base_url+img)
        a = plt.imread(f, format='jpg')
        axes[i].imshow(a)
        axes[i].set_title("{}".format(title))
        axes[i].axis('off')
    plt.show()
    
    candidates = list(map(lambda x: x[0], pred[1]))
    res = es_search(es, candidates)
    
    i = 0
    for prod, slot, prob in pred[1]:
        i+=1
        if prod in res:
            cat = res[prod]['_4']['_5']
            title = res[prod]['_2']
            img = res[prod]['_3']
            #print('{}[{}].{}.prod {}.slot {}.prob {}'.format(title, cat, img, prod, slot, prob))
            #continue
            
            f = urllib.request.urlopen(img_base_url+img)        
            # read the image file in a numpy array
            a = plt.imread(f, format='jpg')
            plt.imshow(a)
            plt.title("{}[{}] score:{} {} [{}]".format(i, slot, prob, title, cat))
            plt.axis('off')
            plt.show()

        else:
            print('no entry for product {}.slot {}.prob {}'.format(prod, slot, prob))

In [8]:
def show_history_only(pred):
    img_base_url=r'http://img.wemep.co.kr/'
    history = es_search(es, pred[0])
        
    fig, axes = plt.subplots(ncols=len(pred[0]), figsize=(16,16))
    
    for i, h in enumerate(pred[0]):
        title = history[h]['_2']
        img = history[h]['_3']
        f = urllib.request.urlopen(img_base_url+img)
        a = plt.imread(f, format='jpg')
        axes[i].imshow(a)
        axes[i].set_title("{}".format(title))
        axes[i].axis('off')
    plt.show()

In [9]:
def show_item_b(item_b, show_figure=False):
    img_base_url=r'http://img.wemep.co.kr/'
    
    candidates = list(map(lambda x: x[0], item_b))
    res = es_search(es, candidates)
    
    i = 0
    for deal, slot, score in item_b:
        i+=1
        if deal in res:
            cat = res[deal]['_4']['_5']
            title = res[deal]['_2']
            img = res[deal]['_3']
            
            f = urllib.request.urlopen(img_base_url+img)        
            # read the image file in a numpy array
            a = plt.imread(f, format='jpg')
            plt.imshow(a)
            plt.title("{}[{}] score:{} {} [{}]".format(i, slot, score, title, cat))
            plt.axis('off')
            plt.show()

        else:
            print('no entry for product {} score{}'.format(deal, score))

In [10]:
def es_scan_extra_by_dids(dids):
    """
    dids로부터, mn, tn1를 가져온다.
    """
    res = es.search(index='dealinfos', 
                body={
                    'from':0, 'size': len(dids),
                    "_source": ["mn", "tn1", "did"],
                    'query': {
                        'ids': {'values': dids }
                        }                        
                    }
               )
    dic = {}
    for hit in res['hits']['hits']:
        dic[hit['_source']['did']] = (hit['_source']['mn'], hit['_source']['tn1'])
    return dic

In [11]:
def es_read_wepick_setting(dt, start_slot=20):
    """
    위픽 세팅 로딩
    """
    res = es.search(index='wepick_setting', 
                body={
                    'query': {
                        'term': {'dt': dt }
                        }                        
                    }
               )
    if res['hits']['total'] > 0:
        dic = {}
        vec = []
        for s in res['hits']['hits'][0]['_source']['setting']:
            row = s.split('_')
            dic[int(row[0])] = int(row[1])
        for slot, did in dic.items():
            if slot >= start_slot:
                vec.append(did)
        return vec, dic
    return None, None

## Ranking 관련

In [12]:
def print_result(predictions, extra):
    for _did, slot, score in predictions:
        did = int(_did)
        if did in extra:
            print((score, did, extra[did][0], slot, extra[did][1]))

In [27]:
def print_predicted_ranks(prediction):
    print_result(prediction, es_scan_extra_by_dids(list(map(lambda x: int(x[0]), prediction))))

In [25]:
def print_item_bias(item_b):
    print_result(item_b, es_scan_extra_by_dids(list(map(lambda x: int(x[0]), item_b))))

In [13]:
# 2018-04-11 21 시의 위픽 세팅 로딩
wepick_setting, wepick_dic = es_read_wepick_setting('2018-04-11 21')

# wepick_pred_a128_32_16_eT

In [29]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model\prediction\wepick_pred_a128_32_16_eT.csv')

In [30]:
item_b = load_item_b(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model\prediction\item_b_pred_a128_32_16_eT.csv')

In [31]:
print_item_bias(item_b)

('1.3208591938018799', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('1.251678466796875', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('1.137528657913208', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.974825918674469', 3525317, '[무료배송] 빅사이즈/원피스/롱티', '28', '원피스')
('0.908465564250946', 3515997, '[블랙데이] 짜장면 치즈 케이크 大', '35', '초콜릿')
('0.7501041889190674', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.6015044450759888', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.5783441066741943', 3525500, '[하객패션] 포커스 봄구성완벽해', '74', '티셔츠')
('0.5189796090126038', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.4662539064884186', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.394110769033432', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.3795723021030426', 3515524, '[무료배송] 롱티/티셔츠/원피스', '45', '티셔츠')
('0.34131279587745667', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.29473137855529785', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.2683102488517761', 3527

In [32]:
# user_history
es_scan_extra_by_dids(pred_base_model['1000007'][0])

{3520987: ('[무료배송] 청바지/밴딩팬츠/스키니', '청바지/진'),
 3526222: ('[무료배송] 리바이스키즈 여름 득템', '유아동신발'),
 3528037: ('[투데이특가] TV화제! 깔라만시 젤리', '캔디/껌/젤리'),
 3532655: ('[무료배송] 팬틴 샴푸×2 + 린스×1', '샴푸/린스'),
 3541064: ('[위메프] 10만 포인트+5,000P', '온라인 이용권')}

In [33]:
# bias
print_predicted_ranks(pred_base_model['1000007'][1])

('0.84', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.81', 3515997, '[블랙데이] 짜장면 치즈 케이크 大', '35', '초콜릿')
('0.79', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.79', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.73', 3525317, '[무료배송] 빅사이즈/원피스/롱티', '28', '원피스')
('0.70', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.68', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.65', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.62', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.62', 3512215, '[6천원쿠폰] 기습쿠폰전 오늘마지막!', '30', '색조메이크업')
('0.60', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.58', 3515524, '[무료배송] 롱티/티셔츠/원피스', '45', '티셔츠')
('0.56', 3521723, '[투데이특가] 샤오미 공기청정기', '25', '공기청정기')
('0.56', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.55', 3527477, '[투데이특가] 니트/가디건/원피스 외', '61', '원피스')
('0.55', 3529131, '휴비딕 체온계 & 탕온도 특가전', '24', '유아건강/유아위생용품')
('0.54', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.54', 3517009, '[투데이특가] 금구운란 30구 1판', '39'

# wepick_pred_a16_32_16_eT

In [40]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model\prediction\wepick_pred_a16_32_16_eT.csv')
item_b = load_item_b(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model\prediction\item_b_pred_a16_32_16_eT.csv')

In [41]:
print_item_bias(item_b)

('1.4381697177886963', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('1.3493484258651733', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('1.1011582612991333', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.9613685011863708', 3525317, '[무료배송] 빅사이즈/원피스/롱티', '28', '원피스')
('0.9165774583816528', 3515997, '[블랙데이] 짜장면 치즈 케이크 大', '35', '초콜릿')
('0.7826852202415466', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.7226204872131348', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.6197749376296997', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.5959551930427551', 3525500, '[하객패션] 포커스 봄구성완벽해', '74', '티셔츠')
('0.581629753112793', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.49305760860443115', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.44319745898246765', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.4296913743019104', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.4122709631919861', 3527477, '[투데이특가] 니트/가디건/원피스 외', '61', '원피스')
('0.3703684806823730

In [42]:
# user_history
es_scan_extra_by_dids(pred_base_model['1000007'][0])

{3520987: ('[무료배송] 청바지/밴딩팬츠/스키니', '청바지/진'),
 3526222: ('[무료배송] 리바이스키즈 여름 득템', '유아동신발'),
 3528037: ('[투데이특가] TV화제! 깔라만시 젤리', '캔디/껌/젤리'),
 3532655: ('[무료배송] 팬틴 샴푸×2 + 린스×1', '샴푸/린스'),
 3541064: ('[위메프] 10만 포인트+5,000P', '온라인 이용권')}

In [43]:
# bias
print_predicted_ranks(pred_base_model['1000007'][1])

('0.83', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.83', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.81', 3515997, '[블랙데이] 짜장면 치즈 케이크 大', '35', '초콜릿')
('0.81', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.73', 3525317, '[무료배송] 빅사이즈/원피스/롱티', '28', '원피스')
('0.71', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.71', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.68', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.65', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.62', 3512215, '[6천원쿠폰] 기습쿠폰전 오늘마지막!', '30', '색조메이크업')
('0.61', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.60', 3521723, '[투데이특가] 샤오미 공기청정기', '25', '공기청정기')
('0.60', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.59', 3527477, '[투데이특가] 니트/가디건/원피스 외', '61', '원피스')
('0.58', 3515524, '[무료배송] 롱티/티셔츠/원피스', '45', '티셔츠')
('0.57', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.57', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.54', 3517009, '[투데이특가] 금구운란 30구 1판', '39', '과자

## wepick_pred_a4_32_16_eT

In [34]:
# 2018-04-11 21시로 특정지어 테스트 중임
pred_base_model = load_pred_result(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model\prediction\wepick_pred_a4_32_16_eT.csv')
item_b = load_item_b(r'c:\Users\wmp\TensorFlow\DeepInterestNetwork\base_model\prediction\item_b_pred_a4_32_16_eT.csv')

In [35]:
print_item_bias(item_b)

('1.4319957494735718', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('1.3778470754623413', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('1.1321226358413696', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('1.0479673147201538', 3515997, '[블랙데이] 짜장면 치즈 케이크 大', '35', '초콜릿')
('0.9421032071113586', 3525317, '[무료배송] 빅사이즈/원피스/롱티', '28', '원피스')
('0.7983036041259766', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.7063745856285095', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.6106392741203308', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.5686817169189453', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.553451657295227', 3525500, '[하객패션] 포커스 봄구성완벽해', '74', '티셔츠')
('0.5316252708435059', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.4269304573535919', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.4131603538990021', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.40746602416038513', 3527477, '[투데이특가] 니트/가디건/원피스 외', '61', '원피스')
('0.36256635189056396

In [36]:
# user_history
es_scan_extra_by_dids(pred_base_model['1000007'][0])

{3520987: ('[무료배송] 청바지/밴딩팬츠/스키니', '청바지/진'),
 3526222: ('[무료배송] 리바이스키즈 여름 득템', '유아동신발'),
 3528037: ('[투데이특가] TV화제! 깔라만시 젤리', '캔디/껌/젤리'),
 3532655: ('[무료배송] 팬틴 샴푸×2 + 린스×1', '샴푸/린스'),
 3541064: ('[위메프] 10만 포인트+5,000P', '온라인 이용권')}

In [37]:
# bias
print_predicted_ranks(pred_base_model['1000007'][1])

('0.81', 3529165, '[하객패션] 락피쉬 18년S/S+20%쿠폰!', '70', '여성단화')
('0.80', 3512593, '[무료배송] 봄 아동복 브랜드 연합전', '36', '아동공용의류')
('0.79', 3528363, '[하객패션] 백화점 잡화 267종! +20%', '27', '벨트')
('0.75', 3515997, '[블랙데이] 짜장면 치즈 케이크 大', '35', '초콜릿')
('0.73', 3525317, '[무료배송] 빅사이즈/원피스/롱티', '28', '원피스')
('0.70', 3514459, '[심야특가] 파파야 여성 의류 모음전', '32', '티셔츠')
('0.68', 3522402, '[무료배송] 에비수 본사특가 20%쿠폰', '44', '티셔츠')
('0.65', 3526985, '[원더쿠폰] 봄 귀걸이,추가할인쿠폰', '42', '귀걸이')
('0.65', 3527575, '[무료배송] 프롬유 ~20%할인쿠폰', '55', '티셔츠')
('0.61', 3544562, '[게릴라특가] 실리콘 주방용품 모음', '40', '조리도구')
('0.59', 3527053, '[투데이특가] 더사랑이 여름 아동복', '43', '아동공용의류')
('0.59', 3527477, '[투데이특가] 니트/가디건/원피스 외', '61', '원피스')
('0.59', 3525068, '[엄마니까] 쁘띠뮤 여름 횡재가격!', '66', '아동공용의류')
('0.58', 3522395, '[롯데] 르까프 아동/성인 빅세일', '46', '남성 티셔츠/상의 기타')
('0.58', 3515524, '[무료배송] 롱티/티셔츠/원피스', '45', '티셔츠')
('0.57', 3525500, '[하객패션] 포커스 봄구성완벽해', '74', '티셔츠')
('0.57', 3539755, '[게릴라특가] 판퍼즐&코코몽자석놀이', '41', '유아퍼즐')
('0.56', 3521723, '[투데이특가] 샤오미 공기청정기', '25', '공기청정기')